In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from keras.preprocessing.image import ImageDataGenerator #Es una clase que nos da keras, tiene metodos que hacen: nos permite recoger imagenes de directorios(mas sencillo), cada categoria un directorio, por cada uno 2 subdirectorios(entreno, validacion)

directorio = '/content/drive/MyDrive/Colab Notebooks/Inteligencia Artificial/Datasets/intel'
train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        rotation_range=10, # rotation
        width_shift_range=0.2, # horizontal shift
        height_shift_range=0.2, # vertical shift
        zoom_range=0.2, # zoom
        horizontal_flip=True, # horizontal flip
        brightness_range=[0.2,1.2]) # brightness
test_datagen = ImageDataGenerator(rescale=1. / 255)


train_generator = train_datagen.flow_from_directory(
    directorio + '/train',
    target_size=(150, 150),#Como el resize
    batch_size=20,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    directorio + '/validation',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

Found 14038 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [9]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense, Dropout

def classification (batch_size = 200, epochs = 20, img_width = 150,    #Si llamo  a la funcion sin argumentos cogera estos por defecto, en otro caso cogera los argumentos especificados y el resto por defecto.
                    img_height = 150, num_train_samples = 14038, 
                    num_validation_samples = 3000):

    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu',
              input_shape=(img_width, img_height, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(48, (3, 3), activation='relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten()) 
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', # Una neurona que clasifica perro o gato
              optimizer='rmsprop',
              metrics=['accuracy'])#Tiene sentido meterlo por q nos da una idea muy intuitiva de lo que estoy haciendo en clasificacion
    
    print(model.summary())
    
    history = model.fit(
        train_generator,
        steps_per_epoch=num_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=num_validation_samples // batch_size, #Los generator sin ules infinitos que t dan valores, entonces de alg8una manera tiene  qsaber cuando cortar keras
        verbose =1)
    
    return history

In [10]:
classification()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 72, 72, 48)        13872     
                                                                 
 dropout_3 (Dropout)         (None, 72, 72, 48)        0         
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 36, 36, 48)       0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 34, 34, 64)       

KeyboardInterrupt: ignored

#No se que hace, ocultar

In [ ]:
from keras.preprocessing.image import ImageDataGenerator#Es una clase que nos da keras, tiene metodos que hacen: nos permite recoger imagenes de directorios(mas sencillo), cada categoria un directorio, por cada uno 2 subdirectorios(entreno, validacion)
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout
from keras.layers import Flatten, Dense
from keras import regularizers

def classification (batch_size = 20, epochs = 20, img_width = 150,
                    img_height = 150, 
                    num_train_samples = 2000, num_validation_samples=800):
    train_data_dir = directorio + '/train'
    validation_data_dir = directorio + '/validation'

    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu',
              input_shape=(img_width, img_height, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(48, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #model.add(Dropout(0.4))      # Dropout
    model.add(Flatten())
    model.add(Dense(64, activation='relu',
              kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dense(1, activation='sigmoid',  ))

    model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

    train_datagen = ImageDataGenerator(rescale=1. / 255)
    test_datagen = ImageDataGenerator(rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),#Como el resize
        batch_size=batch_size,
        class_mode='binary')

    validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

    history = model.fit_generator(
        train_generator,
        steps_per_epoch=num_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=num_validation_samples // batch_size, #Los generator sin ules infinitos que t dan valores, entonces de alg8una manera tiene  qsaber cuando cortar keras
        verbose =0)
    
    return history

history = classification(num_train_samples = 4000)

Found 368 images belonging to 6 classes.
Found 88 images belonging to 6 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:52: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


#Usamos el data transfer en dos etapas

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16

vgg16 = VGG16(weights='imagenet',# 'none' is random'imagenet' is optimized
              include_top = False, # Only the convolutional section
              input_shape = (150,150,3))

train_data_dir = '/content/drive/MyDrive/Colab Notebooks/Inteligencia Artificial/Datasets/intel/train'
validation_data_dir = '/content/drive/MyDrive/Colab Notebooks/Inteligencia Artificial/Datasets/intel/validation'
    
def get_dogsCats_activations_fromVGG16(source_dir, num_features):
    activations = np.zeros((num_features, 4, 4, 512)) # block5_pool shape
    labels = np.zeros((num_features,))
    
    img_generator = ImageDataGenerator(rescale = 1./255.)
    batch_size = 32
    
    generator = img_generator.flow_from_directory(
        train_data_dir,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
        
    i = num_features//batch_size - 1
    for input_batch, label_batch in generator:
        # each image enters in the vgg16 model
        output_batch = vgg16.predict(input_batch)
        # vgg16 exit activations   
        activations[i*batch_size:(i+1)*batch_size] = output_batch  
        labels[i*batch_size:(i+1)*batch_size] = label_batch # labels
        i -= 1
        if i == -1:
            break
            
    return activations, labels
    
train_activations, train_labels =get_dogsCats_activations_fromVGG16(train_data_dir,4096)
validation_activations, validation_labels = get_dogsCats_activations_fromVGG16(validation_data_dir,1024)


Found 14038 images belonging to 6 classes.
1/1 [==============================] - 0s 22ms/step


In [ ]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout

model = Sequential()
model.add(Flatten())
# block5_pool shape (input_dim = 4*4*512) using 150x150 pixels
model.add(Dense(256, activation = 'relu', input_dim = 4*4*512)) 
model.add(Dropout(0.5))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

EPOCHS = 20
history = model.fit(train_activations, train_labels, epochs = EPOCHS,
                    batch_size = 32, validation_data =
                 (validation_activations, validation_labels), verbose = 0)

plot(history)